# HW_01 Рекомендательные системы





hit rate at k

money_precision_at_k

recall_at_k

money_recall_at_k

map_k

* NDCG@k

reciprocal_rank

In [1]:
import pandas as pd
import numpy as np

### 1. Hit rate

Hit rate = был ли хотя бы 1 релевантный товар среди рекомендованных

- Иногда применяется, когда продаются достаточно дрогие товары (например, бытовая техника) 

----
Hit rate = (был ли хотя бы 1 релевантный товар среди рекомендованных)   

Hit rate@k = (был ли хотя бы 1 релевантный товар среди топ-k рекомендованных)

In [2]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

def hit_rate(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate


def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    #print(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate_at_k = (flags.sum() > 0) * 1

    
    return hit_rate_at_k

In [3]:
hit_rate(recommended_list, bought_list)

1

In [4]:
hit_rate_at_k(recommended_list, bought_list, 3)

1

### 2. Precision

*Precision* - доля релевантных товаров среди рекомендованных = Какой % рекомендованных товаров  юзер купил

- Пожалуй, самая приближенная к бизнес-метрикам и самая популярная метрика

---

Precision= (# of recommended items that are relevant) / (# of recommended items)  

Precision@k = (# of recommended items @k that are relevant) / (# of recommended items @k)

Money Precision@k = (revenue of recommended items @k that are relevant) / (revenue of recommended items @k)  

**Note:** Обычно k в precision@k достаточно невелико (5-20) и определяется из бизнес-логики. Например, 5 товаров в e-mail рассылке, 20 ответов на первой странице google и т.д

Красная рыба - 400 руб  
Молоко - 60 руб  
Хлеб = 40 руб  
Гречка = 40 руб  
Шоколад = 90 руб  

------  
Варенье - 240 руб  
...  

**Case 1**  
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [1, 0, 0, 0 , 1]  

$precison@5 = \frac{1 + 0 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{1*400 + 0*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 77.7\%$  

  
**Case 2**   
prices_resommended = [400, 60, 40, 40 , 90]  
flags = [0, 1, 0, 0 , 1]  

$precison@5 = \frac{0 + 1 + 0 +0 + 1}{1+1+1+1+1} = 40\%$  
$money precision@5 = \frac{0*400 + 1*60 + ... + 1*90}{1*400 + 1*60 + ... + 1*90} = 15.8\%$

In [5]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    precision = flags.sum() / len(recommended_list) # делим на все рекоммендации
    
    return precision


def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list) 
    
    precision = flags.sum() / len(recommended_list) 
    
    
    return precision


def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
     
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    prices_recommended = prices_recommended[:k]
    
    flags = np.isin(recommended_list, bought_list) # идем по списку recommended
    
    #print(flags) # длина =k
    #print(prices_recommended) # длина =k
    
    precision = flags[:k].dot(prices_recommended) / sum(prices_recommended) 
    
    
    return precision



In [6]:
precision(recommended_list, bought_list)

0.2

In [7]:
precision_at_k(recommended_list, bought_list, k=5)

0.4

In [8]:
precision_at_k(recommended_list, bought_list, k=3)

0.3333333333333333

In [9]:
prices_recommended = [400, 60, 40, 40 , 90, 60, 70, 55, 70, 90]

money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5)

0.6984126984126984

### 3. Recall

*Recall* - доля рекомендованных товаров среди релевантных = Какой % купленных товаров был среди рекомендованных

- Обычно используется для моделей пре-фильтрации товаров (убрать те товары, которые точно не будем рекомендовать)

---

Recall= (# of recommended items that are relevant) / (# of relevant items)  

Recall@k = (# of recommended items @k that are relevant) / (# of relevant items)

Money Recall@k = (revenue of recommended items @k that are relevant) / (revenue of relevant items)  

    
  
**Note:** в recall@k число k обычно достаточно большое (50-200), больше чем покупок у среднестатистического юзера

In [10]:
def recall(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    recall = flags.sum() / len(bought_list) # делим на релевантных
    
    return recall


def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  
    recommended_list = recommended_list[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    recall = flags.sum() / len(bought_list) # делим на релевантных
    
    return recall


def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    prices_recommended = np.array(prices_recommended)
    prices_bought = np.array(prices_bought)
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]
    prices_recommended = prices_recommended[:k]
    
    flags = np.isin(recommended_list, bought_list) # длина = k
    
    #print(flags)
    #print(prices_recommended)
    
    recall = flags.dot(prices_recommended) / sum(prices_bought)
    
    return recall

In [11]:
recall(recommended_list, bought_list)

0.5

In [12]:
recall_at_k(recommended_list, bought_list, 3)

0.25

In [13]:
prices_bought = [10, 50, 400, 40] # длина списка = длине списка bought_list, цену взять из списка prices_recommended

money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, 3)

0.8

---

# Метрики ранжирования

Если важен порядок рекомендаций. Подробнее можно почитать [здесь](https://habr.com/ru/company/econtenta/blog/303458/). Формулы в статье могут несколько отличаться от формул в лекции 

https://sdsawtelle.github.io/blog/output/mean-average-precision-MAP-for-recommender-systems.html

## AP@k
AP@k - average precision at k

Average Precision (AP) – насколько много релевантных объектов сконцентрировано среди самых высокооцененных

$$AP@k = \frac{1}{r} \sum{[recommended_{relevant_i}] * precision@k}$$

- r - кол-во релевантный среди рекомендованных
- Суммируем по всем релевантным товарам
- Зависит от порядка реокмендаций

In [14]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [15]:
def ap_k(recommended_list, bought_list, k=5):
    
    recommended_list = recommended_list[:k]
    
    score = 0
    num_ = 0

    for i, item in enumerate(recommended_list):
        if item in bought_list and item not in recommended_list[:i]:
            num_ += 1.0
            score += num_ / (i+1)

    return score / min(len(bought_list), k)


In [16]:
ap_k(recommended_list, bought_list, k=5)

0.375

### MAP@k

MAP@k (Mean Average Precision@k)  
Среднее AP@k по всем юзерам
- Показывает средневзвешенную точность рекомендаций

$$MAP@k = \frac{1}{|U|} \sum_u{AP_k}$$
  
|U| - кол-во юзеров

In [17]:
# если входные данные в виде двумерных массивов, складываем ap_k и делим на количество


def map_k(recommended_list, bought_list, k=5):
    
    sum_ap_k = 0
    
    for index in range(len(recommended_list)):
        sum_ap_k += ap_k(recommended_list[index], bought_list[index], k)
        #print(ap_k(recommended_list[index], bought_list[index], k))
    
    return sum_ap_k / len(recommended_list)


In [18]:
recommended_list = [[143, 156, 1134, 991, 27, 1543, 533, 11, 43],
                    [143, 156, 991, 27, 1543, 3345, 533, 11, 43],
                    [143, 156, 1134, 27, 1543, 3345, 533, 11, 43]]
bought_list = [[521, 32, 143, 991, 533],
               [521, 32, 143, 991, 27],
               [521, 32, 143]]

In [19]:
map_k(recommended_list, bought_list, 5)

0.3722222222222222

### NDCG@k
Normalized discounted cumulative gain

In [20]:
# сначала DCG

# на вход получаем список уровней релевантности товаров( 1 - не релевантно, 2 - в целом релевантноб 3 - точное соответствие)
list_rangs = [0, 2, 2, 3, 1, 3]

def dcg(list_rangs):
    score = []
    for i, value in enumerate(list_rangs): 
        a = 2**value - 1
        b =  np.log2(i + 1 + 1) 
        score.append(a/b)
    return sum(score)

In [21]:
dcg(list_rangs)

9.287828284218822

In [22]:
# теперь с нормированием
def ndcg(list_rangs): 
    dcg_ = dcg(list_rangs)
    i_dcg = dcg(sorted(list_rangs, reverse=True))
    ndcg = dcg_ / i_dcg
    return ndcg

In [23]:
ndcg(list_rangs)

0.6363535200392774

[MRR](https://drive.google.com/file/d/1uMFKqWULFmehS0pJtvXKRygjYxQOPise/view?usp=sharing)

In [24]:
recommended_list = [156, 143, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [25]:
def reciprocal_rank(recommended_list, bought_list):
    
    for index in range(len(recommended_list)):
        if recommended_list[index] in bought_list:
            return 1 / (index + 1)
    

In [26]:
reciprocal_rank(recommended_list, bought_list)

0.5